In [ ]:
#
# Simple example of a Julia linear pogramming model
#
 
using LinearAlgebra
using SparseArrays
using Random

const USE_PRE_SOLVE = false 
const USE_INTERIOR_POINT_METHOD = true
const USE_SPARSE_MATRIX = false

# const IS_MINIMIZE::Bool = false
# const MPS_EXAMPLE::String = """
# NAME          EXAMPLE
# 
# OBJSENSE
#  MAX
#  
# ROWS
#  N  COST
#  L  C1
#  L  C2
#  L  C3
# COLUMNS
#     x1        COST       3
#     x1        C1         1
#     x1        C2         1
#     x2        COST       2
#     x2        C1         1
#     x2        C3         1
# RHS
#     RHS       C1         4
#     RHS       C2         2
#     RHS       C3         3
# ENDATA
# """

# https://en.wikipedia.org/wiki/Revised_simplex_method
const IS_MINIMIZE::Bool = false
const MPS_EXAMPLE::String = """
NAME          APPLIED_INTEGER_PROGRAMMING_9_7

OBJSENSE
 MAX

ROWS
 N  OBJ
 L  ROW1
 L  ROW2
 L  ROW3

COLUMNS
    X1        OBJ       4
    X1        ROW1      1
    X1        ROW2      2
    X1        ROW3     -3
    X2        OBJ       3
    X2        ROW1      2
    X2        ROW2     -1
    X2        ROW3      2
    X3        OBJ       1
    X3        ROW1      3
    X3        ROW2      2
    X3        ROW3      1
    X4        OBJ       7
    X4        ROW1      1
    X4        ROW2      2
    X4        ROW3     -1
    X5        OBJ       6
    X5        ROW1     -3
    X5        ROW2      1
    X5        ROW3      2

RHS
    RHS1      ROW1      9
    RHS1      ROW2     10
    RHS1      ROW3     11

BOUNDS
 LO BOUND1    X1        0
 LO BOUND1    X2        0
 LO BOUND1    X3        0
 LO BOUND1    X4        0
 LO BOUND1    X5        0

ENDATA
"""
#=
/opt/homebrew/Cellar/highs/1.7.2/bin/highs --solution_file ex97_highs.txt ex97.mps
Running HiGHS 1.7.2 (git hash: n/a): Copyright (c) 2024 HiGHS under MIT licence terms
LP   ex97 has 3 rows; 5 cols; 15 nonzeros
Coefficient ranges:
  Matrix [1e+00, 3e+00]
  Cost   [1e+00, 7e+00]
  Bound  [0e+00, 0e+00]
  RHS    [9e+00, 1e+01]
Presolving model
3 rows, 5 cols, 15 nonzeros  0s
3 rows, 5 cols, 15 nonzeros  0s
Presolve : Reductions: rows 3(-0); columns 5(-0); elements 15(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.0999978563e+01 Ph1: 3(11); Du: 5(21) 0s
          4     9.4000000000e+01 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 4
Objective value     :  9.4000000000e+01
HiGHS run time      :          0.00

Model status
Optimal

# Primal solution values
Feasible
Objective 94
# Columns 5
X1 7
X2 10
X3 0
X4 0
X5 6
# Rows 3
ROW1 9
ROW2 10
ROW3 11

# Dual solution values
Feasible
# Columns 5
X1 0
X2 0
X3 -16.4285714285714
X4 -2.2380952380952
X5 0
# Rows 3
ROW1 1.4761904761905
ROW2 5.1904761904762
ROW3 2.6190476190476

# Basis
HiGHS v1
Valid
# Columns 5
1 1 0 0 1 
# Rows 3
2 2 2 
=#



# H P Williams examples 2.2
# const IS_MINIMIZE::Bool = false
# const MPS_EXAMPLE::String = """
# NAME          BLEND
# 
# OBJSENSE
#  MAX
#  
# ROWS
#  N  PROF
#  L  VVEG
#  L  NVEG
#  L  UHRD
#  G  LHRD
#  E  CONT
# COLUMNS
#     VEG01     PROF      -110.0    VVEG         1.0
#     VEG01     UHRD         8.8    LHRD         8.8
#     VEG01     CONT         1.0
#     VEG02     PROF      -120.0    VVEG         1.0
#     VEG02     UHRD         6.1    LHRD         6.1
#     VEG02     CONT         1.0
#     OIL01     PROF      -130.0    NVEG         1.0
#     OIL01     UHRD         2.0    LHRD         2.0
#     OIL01     CONT         1.0
#     OIL02     PROF      -110.0    NVEG         1.0
#     OIL02     UHRD         4.2    LHRD         4.2
#     OIL02     CONT         1.0
#     OIL03     PROF      -115.0    NVEG         1.0
#     OIL03     UHRD         5.0    LHRD         5.0
#     OIL03     CONT         1.0
#     PROD      PROF       150.0    UHRD        -6.0
#     PROD      LHRD        -3.0    CONT        -1.0
# RHS
#     RHS       VVEG       200.0
#     RHS       NVEG       250.0
#     RHS       UHRD         0.0
#     RHS       LHRD         0.0
#     RHS       CONT         0.0
# ENDATA
# """

# GLPK example dataset
# const IS_MINIMIZE::Bool = false
# const MPS_EXAMPLE::String = """
# *NAME:         ALLOY
# *ROWS:         22
# *COLUMNS:      20
# *NONZERO:      203
# *OPT SOLN:     2149.247891
# *SOURCE:       Linear Programming--Aluminium Alloy Blending
# *              Data Processing Application. N.Y.: IBM Corp.
# *APPLICATION:  Aluminium Alloy Blending
# *COMMENTS:     fixed MPS format
# *              encoded by Andrew Makhorin <mao@gnu.org>
# *
# NAME          ALLOY
# 
# OBJSENSE
#  MAX
#  
# ROWS
#  N  COST
#  G  ZN  
#  L  ZX  
#  G  CN  
#  L  CX  
#  G  MN  
#  L  MX  
#  G  CHN 
#  L  CHX 
#  G  BN  
#  L  BX  
#  L  IX  
#  L  SX  
#  L  MGX 
#  L  NX  
#  L  TX  
#  L  LX  
#  L  TNX 
#  L  BIX 
#  L  GX  
#  L  SCX 
#  G  FL  
# COLUMNS
# * Pure Aluminium 1
#     A1        COST             .28
#     A1        IX               .0004
#     A1        SX               .0005
#     A1        FL              1.0
# * Pure Aluminium 2
#     A2        COST             .26
#     A2        IX               .0006
#     A2        SX               .0006
#     A2        FL              1.0
# * Pure Aluminium 3
#     A3        COST             .25
#     A3        IX               .0011
#     A3        SX               .0007
#     A3        FL              1.0
# * PuA3 Aluminium 4
#     A4        COST             .23
#     A4        IX               .0026
#     A4        SX               .0012
#     A4        FL              1.0
# * Pure Copper
#     C         COST             .31
#     C         CN              1.00
#     C         CX              1.00
#     C         FL              1.0
# * Pure Magnesium
#     M         COST             .38
#     M         MN              1.00
#     M         MX              1.00
#     M         FL              1.0
# * Beryllium/Aluminium Alloy
#     B/A       COST            3.60
#     B/A       BN              0.0600
#     B/A       BX              0.0600
#     B/A       FL              1.0
# * Pure Zinc
#     Z         COST             .22
#     Z         ZN               .95
#     Z         ZX               .95
#     Z         FL              1.0
# * Chromium Aluminium Alloy
#     C/A       COST             .27
#     C/A       CHN              .0300
#     C/A       CHX              .0300
#     C/A       FL              1.0
# * Scrap 1
#     SC1       COST             .21
#     SC1       ZN               .0009
#     SC1       ZX               .0009
#     SC1       CN               .0444
#     SC1       CX               .0444
#     SC1       MN               .0042
#     SC1       MX               .0042
#     SC1       CHN              .0001
#     SC1       CHX              .0001
#     SC1       IX               .0024
#     SC1       SX               .0101
#     SC1       MGX              .0079
#     SC1       NX               .0001
#     SC1       TX               .0004
#     SC1       LX               .0001
#     SC1       TNX              .0001
#     SC1       GX               .0001
#     SC1       SCX             1.00
#     SC1       FL              1.0
# * Scrap 2
#     SC2       COST             .20
#     SC2       ZN               .0012
#     SC2       ZX               .0012
#     SC2       CN               .0026
#     SC2       CX               .0026
#     SC2       MN               .0060
#     SC2       MX               .0060
#     SC2       CHN              .0018
#     SC2       CHX              .0018
#     SC2       IX               .0026
#     SC2       SX               .0106
#     SC2       MGX              .0003
#     SC2       NX               .0002
#     SC2       TX               .0004
#     SC2       LX               .0001
#     SC2       TNX              .0001
#     SC2       GX               .0002
#     SC2       FL              1.0
# * Scrap 3
#     SC3       COST             .21
#     SC3       ZN               .0568
#     SC3       ZX               .0568
#     SC3       CN               .0152
#     SC3       CX               .0152
#     SC3       MN               .0248
#     SC3       MX               .0248
#     SC3       CHN              .0020
#     SC3       CHX              .0020
#     SC3       IX               .0016
#     SC3       SX               .0013
#     SC3       MGX              .0005
#     SC3       TX               .0004
#     SC3       LX               .0003
#     SC3       TNX              .0003
#     SC3       FL              1.0
# * Scrap 4
#     SC4       COST             .20
#     SC4       ZN               .0563
#     SC4       ZX               .0563
#     SC4       CN               .0149
#     SC4       CX               .0149
#     SC4       MN               .0238
#     SC4       MX               .0238
#     SC4       CHN              .0019
#     SC4       CHX              .0019
#     SC4       IX               .0019
#     SC4       SX               .0011
#     SC4       MGX              .0004
#     SC4       TX               .0004
#     SC4       LX               .0003
#     SC4       TNX              .0003
#     SC4       FL              1.0
# * Scrap 5
#     SC5       COST             .21
#     SC5       ZN               .0460
#     SC5       ZX               .0460
#     SC5       CN               .0071
#     SC5       CX               .0071
#     SC5       MN               .0343
#     SC5       MX               .0343
#     SC5       CHN              .0013
#     SC5       CHX              .0013
#     SC5       IX               .0017
#     SC5       SX               .0013
#     SC5       MGX              .0018
#     SC5       TX               .0002
#     SC5       LX               .0002
#     SC5       TNX              .0002
#     SC5       FL              1.0
# * Scrap 6
#     SC6       COST             .20
#     SC6       ZN               .0455
#     SC6       ZX               .0455
#     SC6       CN               .0071
#     SC6       CX               .0071
#     SC6       MN               .0343
#     SC6       MX               .0343
#     SC6       IX               .0016
#     SC6       SX               .0011
#     SC6       MGX              .0017
#     SC6       TX               .0002
#     SC6       LX               .0002
#     SC6       TNX              .0002
#     SC6       FL              1.0
# * Scrap 7
#     SC7       COST             .21
#     SC7       ZN               .0009
#     SC7       ZX               .0009
#     SC7       CN               .0447
#     SC7       CX               .0447
#     SC7       MN               .0143
#     SC7       MX               .0143
#     SC7       IX               .0026
#     SC7       SX               .0013
#     SC7       MGX              .0052
#     SC7       TX               .0003
#     SC7       LX               .0001
#     SC7       TNX              .0001
#     SC7       FL              1.0
# * Scrap 8
#     SC8       COST             .20
#     SC8       ZN               .0006
#     SC8       ZX               .0006
#     SC8       CN               .0623
#     SC8       CX               .0623
#     SC8       IX               .0017
#     SC8       SX               .0010
#     SC8       MGX              .0025
#     SC8       TX               .0005
#     SC8       LX               .0001
#     SC8       TNX              .0001
#     SC8       GX               .0025
#     SC8       FL              1.0
# * Scrap 9
#     SC9       COST             .21
#     SC9       ZN               .0009
#     SC9       ZX               .0009
#     SC9       CN               .0034
#     SC9       CX               .0034
#     SC9       MN               .0093
#     SC9       MX               .0093
#     SC9       CHN              .0019
#     SC9       CHX              .0019
#     SC9       IX               .0030
#     SC9       SX               .0062
#     SC9       MGX              .0002
#     SC9       TX               .0003
#     SC9       BIX              .0005
#     SC9       FL              1.0
# * Scrap 10
#     SC10      COST             .20
#     SC10       ZN               .0008
#     SC10       ZX               .0008
#     SC10       CN               .0003
#     SC10       CX               .0003
#     SC10       MN               .0249
#     SC10       MX               .0249
#     SC10       CHN              .0016
#     SC10       CHX              .0016
#     SC10       IX               .0015
#     SC10       SX               .0011
#     SC10       MGX              .0002
#     SC10       FL              1.0
# * Scrap 11
#     SC11      COST             .21
#     SC11      ZN               .0675
#     SC11      ZX               .0675
#     SC11      CN               .0195
#     SC11      CX               .0195
#     SC11      MN               .0265
#     SC11      MX               .0265
#     SC11      CHN              .0020
#     SC11      CHX              .0020
#     SC11      IX               .0014
#     SC11      SX               .0008
#     SC11      MGX              .0002
#     SC11      FL              1.0
# RHS
#     RHS       ZN            555.
#     RHS       ZX            590.
#     RHS       CN            140.0
#     RHS       CX            190.0
#     RHS       MN            245.0
#     RHS       MX            275.0
#     RHS       CHN            19.0
#     RHS       CHX            22.0
#     RHS       BN              2.0
#     RHS       BX              4.0
#     RHS       IX             15.0
#     RHS       SX             10.0
#     RHS       MGX             3.0
#     RHS       NX              2.0
#     RHS       TX              2.0
#     RHS       LX              2.0
#     RHS       TNX             2.0
#     RHS       BIX             8.0
#     RHS       GX              8.0
#     RHS       SCX           900.0
#     RHS       FL          10000.
# ENDATA
# """

#=
Running HiGHS 1.7.2 (git hash: n/a): Copyright (c) 2024 HiGHS under MIT licence terms
LP   alloy_simple has 21 rows; 20 cols; 183 nonzeros
Coefficient ranges:
  Matrix [1e-04, 1e+00]
  Cost   [2e-01, 4e+00]
  Bound  [0e+00, 0e+00]
  RHS    [2e+00, 1e+04]
Presolving model
15 rows, 20 cols, 174 nonzeros  0s
10 rows, 20 cols, 122 nonzeros  0s
Presolve : Reductions: rows 10(-11); columns 20(-0); elements 122(-61)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     1.2000400091e+02 Pr: 5(3975.92) 0s
          9     2.1492478910e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 9
Objective value     :  2.1492478910e+03
HiGHS run time      :          0.00
=#

In [ ]:
# Define a struct to represent a Linear Programming problem
struct LPProblem
    is_minimize::Bool  # True if the objective is to minimize
    c::Vector{Float64}  # Objective function coefficients
    A::Matrix{Float64}  # Constraint matrix
    b::Vector{Float64}  # Right-hand side of constraints
    vars::Vector{String}  # Variable names
    constraint_types::Vector{Char}  # Constraint types
end

In [ ]:
function read_mps_from_string(mps_string::String)
    lines = split(mps_string, '\n')
    sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => Dict(), "RHS" => Dict(), "BOUNDS" => Dict())
    current_section = ""
    objective_name = ""
    is_minimize = true

    for line in lines
        words = split(line)
        (isempty(words) || (line[1] == '*')) && continue  # Skip empty lines and comments

        if (line[1] != ' ') && words[1] in ["NAME", "OBJSENSE", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
            current_section = words[1]
            continue
        end

        if current_section == "NAME"
            sections["NAME"] = words[1]
        elseif current_section == "OBJSENSE"
            if words[1] == "MAX"
                is_minimize = false
            end
        elseif current_section == "ROWS"
            row_type, row_name = words
            push!(sections["ROWS"], (type=row_type, name=row_name))
            if row_type == "N"
                objective_name = row_name
            end
        elseif current_section == "COLUMNS"
            col_name, row_name, value = words
            value = parse(Float64, value)
            if !haskey(sections["COLUMNS"], col_name)
                sections["COLUMNS"][col_name] = Dict()
            end
            sections["COLUMNS"][col_name][row_name] = value
        elseif current_section == "RHS"
            if length(words) == 3
                _, row_name, value = words
            else
                row_name, value = words[2:3]
            end
            sections["RHS"][row_name] = parse(Float64, value)
        elseif current_section == "BOUNDS"
            bound_type, _, var_name, value = words
            if !haskey(sections["BOUNDS"], var_name)
                sections["BOUNDS"][var_name] = Dict()
            end
            sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
        end
    end

    # Convert to LPProblem structure
    vars = collect(keys(sections["COLUMNS"]))
    n_vars = length(vars)
    n_constraints = count(row -> row.type != "N", sections["ROWS"])

    c = zeros(n_vars)
    A = zeros(n_constraints, n_vars)
    b = zeros(n_constraints)
    constraint_types = Char[]

    # Populate objective function
    for (i, var) in enumerate(vars)
        if haskey(sections["COLUMNS"][var], objective_name)
            c[i] = sections["COLUMNS"][var][objective_name]
        end
    end

    # Populate constraint matrix and right-hand side
    constraint_index = 0
    for row in sections["ROWS"]
        if row.type != "N"
            constraint_index += 1
            push!(constraint_types, row.type[1])  # Store constraint type
            for (i, var) in enumerate(vars)
                if haskey(sections["COLUMNS"][var], row.name)
                    A[constraint_index, i] = sections["COLUMNS"][var][row.name]
                end
            end
            b[constraint_index] = get(sections["RHS"], row.name, 0.0)
            
            # Adjust for 'G' type constraints
            if row.type == "G"
                A[constraint_index, :] *= -1
                b[constraint_index] *= -1
            end
        end
    end

    # Process bound constraints
    lb = fill(-Inf, n_vars)
    ub = fill(Inf, n_vars)
    for (i, var) in enumerate(vars)
        if haskey(sections["BOUNDS"], var)
            bounds = sections["BOUNDS"][var]
            if haskey(bounds, "LO")
                lb[i] = bounds["LO"]
            end
            if haskey(bounds, "UP")
                ub[i] = bounds["UP"]
            end
            if haskey(bounds, "FX")
                lb[i] = ub[i] = bounds["FX"]
            end
        else
            lb[i] = 0.0  # Default lower bound is 0 if not specified
        end
    end

    # Add bound constraints to A and b
    n_bound_constraints = count(x -> x > -Inf, lb) + count(x -> x < Inf, ub)
    A_with_bounds = zeros(n_constraints + n_bound_constraints, n_vars)
    b_with_bounds = zeros(n_constraints + n_bound_constraints)
    
    A_with_bounds[1:n_constraints, :] = A
    b_with_bounds[1:n_constraints] = b
    
    bound_constraint_index = n_constraints
    for i in 1:n_vars
        if lb[i] > -Inf
            bound_constraint_index += 1
            A_with_bounds[bound_constraint_index, i] = 1
            b_with_bounds[bound_constraint_index] = lb[i]
            push!(constraint_types, 'G')
        end
        if ub[i] < Inf
            bound_constraint_index += 1
            A_with_bounds[bound_constraint_index, i] = 1
            b_with_bounds[bound_constraint_index] = ub[i]
            push!(constraint_types, 'L')
        end
    end

    return LPProblem(is_minimize, c, A_with_bounds, b_with_bounds, vars, constraint_types)
end

In [ ]:
function presolve(lp::LPProblem; eps::Float64=1e-8)
    is_minimize, c, A, b = lp.is_minimize, lp.c, sparse(lp.A), lp.b
    vars, constraint_types = lp.vars, lp.constraint_types
    m, n = size(A)

    # Initialize masks for rows and columns to keep
    keep_rows = trues(m)
    keep_cols = trues(n)

    # Step 1: Remove zero columns
    for j in 1:n
        if nnz(A[:, j]) == 0 && abs(c[j]) < eps
            keep_cols[j] = false
        end
    end

    # Step 2: Remove zero rows
    for i in 1:m
        if nnz(A[i, :]) == 0
            if abs(b[i]) < eps
                keep_rows[i] = false
            else
                error("Infeasible problem: zero row with non-zero RHS")
            end
        end
    end

    # Step 3: Remove duplicate rows
    for i in 1:m
        if !keep_rows[i]
            continue
        end
        for j in (i+1):m
            if !keep_rows[j]
                continue
            end
            if A[i, :] ≈ A[j, :] && abs(b[i] - b[j]) < eps && constraint_types[i] == constraint_types[j]
                keep_rows[j] = false
            elseif A[i, :] ≈ -A[j, :] && abs(b[i] + b[j]) < eps && 
                   ((constraint_types[i] == '≤' && constraint_types[j] == '≥') || 
                    (constraint_types[i] == '≥' && constraint_types[j] == '≤'))
                keep_rows[j] = false
            end
        end
    end

    # Step 4: Fix variables and tighten bounds
    fixed_vars = Dict{String, Float64}()
    for j in 1:n
        col = A[:, j]
        if nnz(col) == 1
            i = findfirst(!iszero, col)
            if abs(col[i]) ≈ 1 && constraint_types[i] == '='
                val = b[i] / col[i]
                fixed_vars[vars[j]] = val
                keep_cols[j] = false
                b .-= val * col
            end
        end
    end

    # Apply the reductions
    A_new = A[keep_rows, keep_cols]
    b_new = b[keep_rows]
    c_new = c[keep_cols]
    vars_new = vars[keep_cols]
    constraint_types_new = constraint_types[keep_rows]

    # Adjust the objective for fixed variables
    obj_adjust = isempty(fixed_vars) ? 0.0 : sum(c[j] * val for (j, val) in enumerate(lp.vars) if haskey(fixed_vars, val))

    println("Presolve summary:")
    println("Original problem size: $(m) x $(n)")
    println("Reduced problem size: $(sum(keep_rows)) x $(sum(keep_cols))")
    println("Number of fixed variables: $(length(fixed_vars))")
    println("Objective adjustment: $obj_adjust")

    return LPProblem(is_minimize, c_new, Matrix(A_new), b_new, vars_new, constraint_types_new), fixed_vars, obj_adjust
end

In [ ]:
#=
Theoretical basis for each step:

Problem Formulation:

The Revised Simplex Method solves linear programming problems in the standard form:
Maximize c^T x
Subject to Ax = b
x ≥ 0


Slack Variables (Step 0):

Slack variables are added to convert inequality constraints to equality constraints.
This ensures that the problem is in standard form for the simplex method.


Basic and Non-Basic Variables:

The method partitions variables into basic (B) and non-basic (N) variables.
Initially, slack variables form the basis, corresponding to the identity matrix in the augmented constraint matrix.


Revised Simplex Iteration:
a. Basic Solution (Step 1):

Compute x_B = B^(-1) * b, where B is the basis matrix.
This gives the values of basic variables in the current solution.

b. Reduced Costs (Step 2):

Compute y = c_B' * B^(-1), where c_B are the objective coefficients of basic variables.
Calculate reduced costs: c_N - y' * A_N, where A_N are columns of non-basic variables.
Reduced costs measure the rate of change in the objective for unit increase in non-basic variables.

c. Optimality Check (Step 3):

If all reduced costs are non-positive, the current solution is optimal.
This is based on the theorem that a basic feasible solution is optimal if and only if all reduced costs are non-positive.

d. Entering Variable (Step 4):

Choose the non-basic variable with the most positive reduced cost to enter the basis.
This variable has the potential to improve the objective value the most.

e. Direction Computation (Step 5):

Compute d = B^(-1) * A_q, where A_q is the column of the entering variable.
This determines how basic variables change as the entering variable increases.

f. Unboundedness Check (Step 6):

If all elements of d are non-positive and the reduced cost is positive, the problem is unbounded.
This means we can increase the entering variable indefinitely, improving the objective without bound.

g. Leaving Variable (Step 7):

Perform the minimum ratio test: min(x_B_i / d_i) for d_i > 0.
This determines how far we can move along the edge without violating non-negativity constraints.

h. Basis Update (Step 8):

Swap the entering and leaving variables in the basis.
This moves to an adjacent extreme point of the feasible region.


Convergence:

The algorithm repeats these steps until optimality is reached or unboundedness is detected.
Each iteration either improves the objective value or determines that the problem is unbounded.
For non-degenerate problems, the algorithm is guaranteed to terminate in a finite number of steps.


Numerical Considerations:

Small tolerance (1e-10) is used for numerical stability in comparisons.
A maximum iteration limit prevents infinite loops in case of cycling (rare in practice, but possible in degenerate problems).

This implementation of the Revised Simplex Method efficiently solves linear programming problems by working with the inverse of the basis matrix and reduced costs, rather than maintaining the entire tableau as in the standard Simplex Method.
=#

function revised_simplex(lp::LPProblem)
    c, A, b = lp.c, lp.A, lp.b
    m, n = size(A)
    
    # If minimizing, negate the objective function to convert to a maximization problem
    if lp.is_minimize
        c = -c
    end
    
    println("\nInitial problem:")
    println("Objective function coefficients c: ", lp.c)
    println("Constraint matrix A: ", lp.A)
    println("Right-hand side b: ", lp.b)
    println("Variables: ", lp.vars) 
    println("Optimization type: ", lp.is_minimize ? "Minimize" : "Maximize")
    
    # Step 0: Add slack variables to convert inequalities to equalities
    A = [A I(m)]  # Augment A with identity matrix for slack variables
    c = [c; zeros(m)]  # Extend objective coefficients with zeros for slack variables
    n = n + m  # Update number of variables
    
    # Initialize basis with slack variables
    B = collect(n-m+1:n)  # Indices of basic variables
    N = collect(1:n-m)    # Indices of non-basic variables
    
    println("Initial basis: ", B)
    println("Initial non-basic variables: ", N)
    
    iteration = 0
    while true
        iteration += 1
        println("\nIteration ", iteration)
        
        # Step 1: Compute basic solution
        B_matrix = A[:, B]  # Extract basis matrix
        x_B = B_matrix \ b  # Solve B * x_B = b for basic variables
        
        println("Basic solution: ", x_B)
        
        # Step 2: Compute reduced costs
        y = (c[B]' / B_matrix)'  # Compute dual variables: y' * B = c_B'
        c_N = c[N] - A[:, N]' * y  # Compute reduced costs for non-basic variables
        
        println("Reduced costs: ", c_N)
        
        # Step 3: Check optimality
        if all(c_N .<= 1e-10)  # If all reduced costs are non-positive, solution is optimal
            x = zeros(n)
            x[B] = x_B
            println("Optimal solution found")
            obj_value = dot(c[1:length(lp.vars)], x[1:length(lp.vars)])
            if lp.is_minimize
                obj_value = -obj_value  # Convert back to minimization objective
            end
            return x[1:length(lp.vars)], obj_value
        end
        
        # Step 4: Choose entering variable (most positive reduced cost)
        e = argmax(c_N)
        q = N[e]
        
        println("Entering variable: ", q)
        
        # Step 5: Compute direction of edge to traverse
        d = B_matrix \ A[:, q]
        
        println("Direction: ", d)
        
        # Step 6: Check unboundedness
        if all(d .<= 1e-10)  # If direction is non-positive, problem is unbounded
            error("Problem is unbounded")
        end
        
        # Step 7: Choose leaving variable (minimum ratio test)
        ratios = x_B ./ d
        ratios[d .<= 1e-10] .= Inf  # Avoid division by zero or negative values
        l = argmin(filter(x -> x > 0, ratios))
        
        p = B[l]
        println("Leaving variable: ", p)
        
        # Step 8: Update basis
        B[l] = q
        N[e] = p
        
        println("New basis: ", B)
        println("New non-basic variables: ", N)
        
        # Safeguard against infinite loops
        if iteration > 100
            println("Maximum iterations reached")
            break
        end
    end
    
    error("Algorithm did not converge")
end

In [ ]:
function revised_simplex_sparse(lp::LPProblem)
    c, A, b = lp.c, sparse(lp.A), lp.b
    m, n = size(A)
    
    # If minimizing, negate the objective function to convert to a maximization problem
    if lp.is_minimize
        c = -c
    end
    
    println("\nInitial problem:")
    println("Objective function coefficients c: ", lp.c)
    println("Constraint matrix A: ", lp.A)
    println("Right-hand side b: ", lp.b)
    println("Variables: ", lp.vars)
    println("Optimization type: ", lp.is_minimize ? "Minimize" : "Maximize")
    
    # Step 0: Add slack variables to convert inequalities to equalities
    A = [A sparse(I, m, m)]  # Augment A with sparse identity matrix for slack variables
    c = [c; zeros(m)]  # Extend objective coefficients with zeros for slack variables
    n = n + m  # Update number of variables
    
    # Initialize basis with slack variables
    B = collect(n-m+1:n)  # Indices of basic variables
    N = collect(1:n-m)    # Indices of non-basic variables
    
    println("Initial basis: ", B)
    println("Initial non-basic variables: ", N)
    
    iteration = 0
    while true
        iteration += 1
        println("\nIteration ", iteration)
        
        # Step 1: Compute basic solution
        B_matrix = A[:, B]  # Extract basis matrix
        x_B = B_matrix \ Vector(b)  # Solve B * x_B = b for basic variables
        
        println("Basic solution: ", x_B)
        
        # Step 2: Compute reduced costs
        y = (Vector(c[B])' / B_matrix)'  # Compute dual variables: y' * B = c_B'
        c_N = c[N] - A[:, N]' * y  # Compute reduced costs for non-basic variables
        
        println("Reduced costs: ", c_N)
        
        # Step 3: Check optimality
        if all(c_N .<= 1e-10)  # If all reduced costs are non-positive, solution is optimal
            x = spzeros(n)
            x[B] = x_B
            println("Optimal solution found")
            obj_value = dot(c[1:length(lp.vars)], x[1:length(lp.vars)])
            if lp.is_minimize
                obj_value = -obj_value  # Convert back to minimization objective
            end
            return Array(x[1:length(lp.vars)]), obj_value
        end
        
        # Step 4: Choose entering variable (most positive reduced cost)
        e = argmax(c_N)
        q = N[e]
        
        println("Entering variable: ", q)
        
        # Step 5: Compute direction of edge to traverse
        d = B_matrix \ Vector(A[:, q])  # Convert sparse column to dense vector
        
        println("Direction: ", d)
        
        # Step 6: Check unboundedness
        if all(d .<= 1e-10)  # If direction is non-positive, problem is unbounded
            error("Problem is unbounded")
        end
        
        # Step 7: Choose leaving variable (minimum ratio test)
        ratios = x_B ./ d
        ratios[d .<= 1e-10] .= Inf  # Avoid division by zero or negative values
        l = argmin(filter(x -> x > 0, ratios))
        
        p = B[l]
        println("Leaving variable: ", p)
        
        # Step 8: Update basis
        B[l] = q
        N[e] = p
        
        println("New basis: ", B)
        println("New non-basic variables: ", N)
        
        # Safeguard against infinite loops
        if iteration > 100
            println("Maximum iterations reached")
            break
        end
    end
    
    error("Algorithm did not converge")
end

In [ ]:
function safe_divide(a, b, eps=1e-15)
    return a ./ (b .+ eps)
end

In [ ]:
function solve_kkt_system(A, G, c, b, h, x, s, y, z, mu)
    m, n = size(A)
    p = size(G, 1)
    
    rc = A'*y + G'*z + s - c
    rb = A*x - b
    rh = G*x - h - z
    rxs = x .* s .- mu
    rzh = z .* (G*x - h) .- mu
    
    if p > 0
        D = Diagonal(safe_divide(1, s) + safe_divide(1, G*x - h))
        M = [A'*A + G'*D*G A'; A zeros(m,m)]
        rhs = [-(rc - G'*safe_divide(rzh, G*x - h) - safe_divide(rxs, x));
               -rb]
    else
        D = Diagonal(safe_divide(1, s))
        M = [A'*A A'; A zeros(m,m)]
        rhs = [-(rc - safe_divide(rxs, x));
               -rb]
    end
    
    sol = M \ rhs
    dx = sol[1:n]
    dy = sol[n+1:end]
    
    if p > 0
        dz = safe_divide(rzh - z.*(G*dx), G*x - h)
    else
        dz = Float64[]
    end
    ds = safe_divide(rxs - s.*dx, x)
    
    return dx, ds, dy, dz
end

In [ ]:
function primal_dual_infeasible_ipm(c, A, b, G, h; max_iter=10000, tol=1e-8)
    m, n = size(A)
    p = size(G, 1)
    
    # Initialize variables
    x = ones(n)
    s = ones(n)
    y = zeros(m)
    z = p > 0 ? ones(p) : Float64[]
    
    for k in 1:max_iter
        # Compute mu and residuals
        mu = (dot(x, s) + (p > 0 ? dot(z, G*x - h) : 0)) / (n + p)
        primal_infeas = norm(A*x - b) / (1 + norm(b))
        dual_infeas = norm(A'*y + G'*z + s - c) / (1 + norm(c))
        gap = abs(dot(c, x) - dot(b, y) - (p > 0 ? dot(h, z) : 0)) / (1 + abs(dot(c, x)))
        
        # Check for convergence
        if primal_infeas < tol && dual_infeas < tol && gap < tol
            return x, dot(c, x)
        end
        
        # Compute search direction
        dx, ds, dy, dz = solve_kkt_system(A, G, c, b, h, x, s, y, z, mu)
        
        # Compute step size
        alpha_p = 0.99 * min(1, minimum(safe_divide(-x, dx)))
        alpha_d = 0.99 * min(1, minimum(safe_divide(-s, ds)))
        if p > 0
            alpha_p = min(alpha_p, 0.99*minimum(safe_divide(h-G*x, G*dx)))
            alpha_d = min(alpha_d, 0.99*minimum(safe_divide(-z, dz)))
        end
        alpha = min(alpha_p, alpha_d)
        
        # Update variables
        x += alpha * dx
        s += alpha * ds
        y += alpha * dy
        if p > 0
            z += alpha * dz
        end
        
        if k % 100 == 0
            println("Iteration $k:")
            println("Primal infeasibility: ", primal_infeas)
            println("Dual infeasibility: ", dual_infeas)
            println("Duality gap: ", gap)
            println("Objective value: ", dot(c, x))
        end
    end
    
    error("Maximum iterations reached without convergence")
end

In [ ]:
function interior_point_method(lp::LPProblem; max_iter=10000, tol=1e-8)
    c, A, b = lp.c, lp.A, lp.b
    m, n = size(A)
    
    println("\nInitial problem:")
    println("Objective function coefficients c: ", lp.c)
    println("Constraint matrix A: ", lp.A)
    println("Right-hand side b: ", lp.b)
    println("Variables: ", lp.vars)
    println("Optimization type: ", lp.is_minimize ? "Minimize" : "Maximize")    
    
    # Convert inequality constraints to standard form
    G = spzeros(0, n)
    h = Float64[]
    new_A = similar(A, 0, n)
    new_b = Float64[]
    for (i, type) in enumerate(lp.constraint_types)
        if type == '≤'
            G = [G; A[i,:]']
            push!(h, b[i])
        elseif type == '≥'
            G = [G; -A[i,:]']
            push!(h, -b[i])
        else  # type == '='
            new_A = [new_A; A[i,:]']
            push!(new_b, b[i])
        end
    end
    
    # Solve using primal-dual infeasible interior point method
    x, obj_value = primal_dual_infeasible_ipm(c, new_A, new_b, G, h, max_iter=max_iter, tol=tol)
    
    return x, obj_value
end

In [ ]:
# Example usage
lp = read_mps_from_string(MPS_EXAMPLE)

# Presolve the problem
if USE_PRE_SOLVE
    lp, fixed_vars, obj_adjust = presolve(lp)

    println("\nFixed variables:")
    for (var, val) in fixed_vars
        println("$var = $val")
    end

    println("\nReduced problem:")
    println("Minimize: $(lp.is_minimize)")
    println("c = $(lp.c)")
    println("A = $(lp.A)")
    println("b = $(lp.b)")
    println("vars = $(lp.vars)")
    println("constraint_types = $(lp.constraint_types)")

end

# solve the LP problem

if USE_INTERIOR_POINT_METHOD
        println("\nSolving the reduced problem using the Interior Point Method with dense matrices")
        execution_time = @elapsed x, obj_value = interior_point_method(lp)    
else
    if USE_SPARSE_MATRIX
        println("\nSolving the reduced problem using the Revised Simplex Method with sparse matrices")
        execution_time = @elapsed x, obj_value = revised_simplex_sparse(lp)
    else
        println("\nSolving the reduced problem using the Revised Simplex Method with dense matrices")
        execution_time = @elapsed x, obj_value = revised_simplex(lp)
    end
end
println("\nExecution time: $execution_time seconds")

# show the solution

println("\nFinal solution:")
for (var, val) in zip(lp.vars, x[1:length(lp.vars)])
    println("$var = $(round(val, digits=6))")
end
println("Objective value: $(round(obj_value, digits=6))")